In [9]:
# Which states have more/less monitors? (Rank states!)
# para responder é necesário contar todas as posições diferenciadas (lat long) para cada estado
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from time import process_time,perf_counter
t_start,r_start=process_time(),perf_counter()
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
    lines = sc.textFile('epa_hap_daily_summary-small.csv') #load txt
    header=lines.first()
    l2 = lines.filter( lambda line : line !=header) #filtra apenas linhas maiores que 0
    l3 = l2.map( lambda line : line.split(',') ) # faz o split das linhas num array
    logRows = l3.map( lambda arr : Row( state = arr[24], mon_pos = arr[5]+arr[6])) #map rdd com colonas nomeadas 
    DFsql = spark.createDataFrame( logRows ) # transforma rdd em dataframe
    DFsql.createOrReplaceTempView("log")
    query_1 = spark.sql("SELECT state, size(collect_set(mon_pos)) AS number_monitors FROM log GROUP BY state ORDER BY number_monitors DESC")
    #query_1=spark.sql('SELECT state,DISTINCT COUNT(mon_pos) AS monitors FROM log GROUP BY state ORDER BY monitors DESC')
    query_1.show(100)
    t_stop,r_stop=process_time(),perf_counter()
    print(f'process time, real time {t_stop-t_start,r_stop-r_start}')
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+--------------------+---------------+
|               state|number_monitors|
+--------------------+---------------+
|          California|            170|
|               Texas|            133|
|           Minnesota|             94|
|            Michigan|             92|
|                Ohio|             91|
|            New York|             67|
|      South Carolina|             64|
|             Montana|             62|
|        Pennsylvania|             61|
|             Florida|             55|
|             Indiana|             52|
|            Colorado|             51|
|      North Carolina|             50|
|            Illinois|             49|
|          Washington|             43|
|           Louisiana|             41|
|             Arizona|             38|
|              Kansas|             37|
|             Georgia|             35|
|            Kentucky|             34|
|              Oregon|             32|
|             Alabama|             31|
|           Tennessee|   

In [10]:
# Which states have more/less monitors? (Rank states!) com read.option na criação dataframe
# para responder é necesário contar todas as posições diferenciadas (lat long) para cada estado
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from time import process_time,perf_counter
t_start,r_start=process_time(),perf_counter()
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
    main_data=spark.read.option('header','true').csv('epa_hap_daily_summary-small.csv')
    main_data.createOrReplaceTempView("log")
    #query_1 = spark.sql("SELECT state, size(collect_set(mon_pos)) AS number_monitors FROM log GROUP BY state ORDER BY number_monitors DESC")
    query_1=spark.sql('SELECT state_name, COUNT(DISTINCT latitude,longitude) as number_monitors FROM log GROUP BY state_name ORDER BY number_monitors DESC')
    query_1.show(100)
    t_stop,r_stop=process_time(),perf_counter()
    print(f'process time, real time {t_stop-t_start,r_stop-r_start}')
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+--------------------+---------------+
|          state_name|number_monitors|
+--------------------+---------------+
|          California|            170|
|               Texas|            133|
|           Minnesota|             94|
|            Michigan|             92|
|                Ohio|             91|
|            New York|             67|
|      South Carolina|             64|
|             Montana|             62|
|        Pennsylvania|             61|
|             Florida|             55|
|             Indiana|             52|
|            Colorado|             51|
|      North Carolina|             50|
|            Illinois|             49|
|          Washington|             43|
|           Louisiana|             41|
|             Arizona|             38|
|              Kansas|             37|
|             Georgia|             35|
|            Kentucky|             34|
|              Oregon|             32|
|             Alabama|             31|
|           Tennessee|   

In [11]:
#Which counties have the best/worst air quality? (Rank counties considering pollutants’level
#linha indice 16 media de valores por dia soma(media diarias)/numero de dias 
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from time import process_time,perf_counter
t_start,r_start=process_time(),perf_counter()
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
    lines = sc.textFile('epa_hap_daily_summary-small.csv') #load txt
    header=lines.first()
    l2 = lines.filter( lambda line : line !=header) #filtra apenas linhas maiores que 0
    l3 = l2.map( lambda line : line.split(',') ) # faz o split das linhas num array
    logRows = l3.map( lambda arr : Row( counties = arr[25], media_pol = arr[16])) #map rdd com colunas nomeadas
    DFsql = spark.createDataFrame( logRows ) # transforma rdd em dataframe
    DFsql.createOrReplaceTempView("log")
    query_2=spark.sql('SELECT counties, AVG(media_pol) AS media_poluicao FROM log GROUP BY counties ORDER BY media_poluicao DESC')
    query_2.show(100)
    t_stop,r_stop=process_time(),perf_counter()
    print(f'process time, real time {t_stop-t_start,r_stop-r_start}')
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+--------------------+------------------+
|            counties|    media_poluicao|
+--------------------+------------------+
|              Tipton|            2556.0|
|              Nassau|              19.0|
|          Columbiana| 7.385690735785953|
|     CHIHUAHUA STATE|         4.5121875|
|            Caldwell| 4.116666666666667|
|              Madera|            3.7393|
|             Oakland| 2.888877848101266|
|               Duval|2.7794603978494625|
|              Kearny|2.3753333333333333|
|               Bucks|2.3674999999999997|
|     San Luis Obispo|2.3333333333333335|
|           Edgecombe|             2.325|
|              Pawnee|2.2941176470588234|
|         Westchester|          2.239375|
|            Johnston|             2.225|
|            Hartford|2.0787055896226416|
|           Granville|2.0285714285714285|
|              Asotin|             2.025|
|              Duplin|               2.0|
|             Boulder| 1.960470901234568|
|          Crittenden|1.9000000000

In [12]:
#Q.3) Which states have the best/worst air quality in each year? (Rank states per year considering pollutants’ level!
# linha 12 contem data
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from time import process_time,perf_counter
t_start,r_start=process_time(),perf_counter()
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
    lines = sc.textFile('epa_hap_daily_summary-small.csv') #load txt
    header=lines.first()
    l2 = lines.filter( lambda line : line !=header) #filtra apenas linhas maiores que 0
    l3 = l2.map( lambda line : line.split(',') ) # faz o split das linhas num array
    logRows = l3.map( lambda arr : Row( states = arr[24],data=arr[11], media_pol = arr[16])) #map rdd com colonas nomeadas 
    DFsql = spark.createDataFrame( logRows ) # transforma rdd em dataframe
    DFsql.createOrReplaceTempView("log")
    query_3=spark.sql('SELECT SUBSTRING(data,0,4) AS ano, states, AVG(media_pol) AS media FROM log GROUP BY ano,states ORDER BY ano,media') 
    query_3.show(100)
    t_stop,r_stop=process_time(),perf_counter()
    print(f'process time, real time {t_stop-t_start,r_stop-r_start}')    
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+----+--------------------+--------------------+
| ano|              states|               media|
+----+--------------------+--------------------+
|1990|            Oklahoma|                 0.0|
|1990|           Wisconsin|                 0.0|
|1990|      Virgin Islands|                 0.0|
|1990|       West Virginia|                 0.0|
|1990|              Hawaii|1.970370370370370...|
|1990|              Nevada|4.208000000000000...|
|1990|              Alaska|4.420833333333333...|
|1990|        South Dakota|            5.705E-4|
|1990|          Washington|5.974999999999999E-4|
|1990|             Wyoming|6.045454545454545E-4|
|1990|                Utah|7.970588235294118E-4|
|1990|          New Mexico|8.222222222222222E-4|
|1990|              Oregon|8.596296296296297E-4|
|1990|             Arizona|8.620134228187919E-4|
|1990|               Maine|9.789285714285713E-4|
|1990|            Colorado|0.002162374100719...|
|1990|         Mississippi|0.002666666666666...|
|1990|            Mi

In [13]:
#Q.4) For each state, what is the average distance (in km) of the monitors in that state to the state center? 
#For simplicity, assume that 1 degree of latitude or logitude equals to 111 km. (Monitor dispersion per state!)
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import math
from time import process_time,perf_counter
t_start,r_start=process_time(),perf_counter()
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
    #load data
    main_data=spark.read.option('header','true').csv('epa_hap_daily_summary-small.csv')#.select(col('state_name'),col('latitude'),col('longitude')).distinct() #data principal
    aux_data=spark.read.option('header','true').csv('usa_states.csv')#.select(col('Name'), col('MinLat'),col('MaxLat'),col('MinLon'),col('MaxLon')) #data auxiliar
    #create tempviews SQL
    main_data.createOrReplaceTempView('maindata')
    aux_data.createOrReplaceTempView('auxdata')
    #start sql processing 
    monitors=spark.sql('SELECT DISTINCT state_name,latitude,longitude FROM maindata') #get distinct monitors
    state_center=spark.sql('SELECT Name,(MinLat+MaxLat)/2 AS lat_center,(MinLon+MaxLon)/2 AS lon_center FROM auxdata') #get state center
    # create new tempviews
    monitors.createOrReplaceTempView('monitors')
    state_center.createOrReplaceTempView('state_center')
    #join tables
    joint_data=spark.sql('SELECT monitors.state_name,monitors.latitude,monitors.longitude, state_center.lat_center,state_center.lon_center FROM monitors LEFT JOIN state_center ON monitors.state_name=state_center.Name')
    joint_data.createOrReplaceTempView('full_data')
    query=spark.sql('SELECT state_name,AVG(SQRT(POWER((latitude-lat_center)*111,2)+POWER((longitude-lon_center)*111,2))) AS dist FROM full_data GROUP BY state_name ORDER BY dist DESC')
    t_stop,r_stop=process_time(),perf_counter()
    print(f'process time, real time {t_stop-t_start,r_stop-r_start}')
    query.show(500)
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

process time, real time (0.24987970000000104, 1.7246517001185566)
+--------------------+------------------+
|          state_name|              dist|
+--------------------+------------------+
|            Virginia| 715.4329730260437|
|              Alaska| 603.6996422410685|
|               Texas|  512.183989163014|
|             Vermont|504.06323548508084|
|            Illinois|440.85401432128594|
|        South Dakota|365.84513743232617|
|             Florida| 336.5449146570832|
|          California|328.22638131553657|
|            Michigan| 326.4116064851129|
|              Nevada|326.28118071973904|
|            Nebraska| 307.1411826055287|
|              Kansas| 292.0796841296708|
|               Idaho|289.63507327563104|
|             Montana| 286.8383352756942|
|            New York| 283.7273398637172|
|             Wyoming| 283.6405863374747|
|              Oregon| 268.8538079232644|
|        Pennsylvania|251.41517634057166|
|        North Dakota|  248.421930732627|
|         

In [14]:
#Q.5) How many sensors there are per quadrant (NW, NE, SE, SW) in each state? To answer this question, you should approximate each state’s area to a rectangle as
#defined in the file “usa_satates.csv”, and divide that area in 4 quadrants (NW,NE, SE, SW). (Count monitors per sate qudrant!

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import math
from time import process_time,perf_counter
t_start,r_start=process_time(),perf_counter()
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
    #load data
    main_data=spark.read.option('header','true').csv('epa_hap_daily_summary-small.csv')#.select(col('state_name'),col('latitude'),col('longitude')).distinct() #data principal
    aux_data=spark.read.option('header','true').csv('usa_states.csv')#.select(col('Name'), col('MinLat'),col('MaxLat'),col('MinLon'),col('MaxLon')) #data auxiliar
    #create tempviews SQL
    main_data.createOrReplaceTempView('maindata')
    aux_data.createOrReplaceTempView('auxdata')
    #start sql processing 
    monitors=spark.sql('SELECT DISTINCT state_name,latitude,longitude FROM maindata') #get distinct monitors
    state_center=spark.sql('SELECT Name,(MinLat+MaxLat)/2 AS lat_center,(MinLon+MaxLon)/2 AS lon_center FROM auxdata') #get state center
    # create new tempviews
    monitors.createOrReplaceTempView('monitors')
    state_center.createOrReplaceTempView('state_center')
    #join tables
    joint_data=spark.sql('SELECT monitors.state_name,monitors.latitude,monitors.longitude, state_center.lat_center,state_center.lon_center FROM monitors LEFT JOIN state_center ON monitors.state_name=state_center.Name')
    joint_data.createOrReplaceTempView('full_data')
    organize=spark.sql('SELECT state_name,CASE WHEN latitude>lat_center and longitude>lon_center THEN 1 ELSE 0 END AS NE,CASE WHEN latitude<lat_center and longitude>lon_center THEN 1 ELSE 0 END AS SE,CASE WHEN latitude<lat_center and longitude<lon_center THEN 1 ELSE 0 END AS SW,CASE WHEN latitude>lat_center and longitude<lon_center THEN 1 ELSE 0 END AS NW FROM full_data')
    organize.createOrReplaceTempView('mon_quad')
    query=spark.sql('SELECT state_name,SUM(NE) AS NE,SUM(SE) AS SE,SUM(SW) AS SW,SUM(NW) AS MW FROM mon_quad GROUP BY state_name') 
    t_stop,r_stop=process_time(),perf_counter()
    print(f'process time, real time {t_stop-t_start,r_stop-r_start}')   
    query.show(500)
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

process time, real time (0.09448460000000125, 1.6038611999247223)
+--------------------+---+---+---+---+
|          state_name| NE| SE| SW| MW|
+--------------------+---+---+---+---+
|                Utah|  0|  3|  3|  6|
|              Hawaii|  2|  1|  0|  2|
|   Country Of Mexico|  0|  0|  0|  0|
|           Minnesota| 11| 50| 21| 12|
|                Ohio| 30| 10| 36| 15|
|              Oregon|  3|  1| 13| 15|
|            Arkansas|  2|  1|  5|  3|
|District Of Columbia|  0|  1|  3|  1|
|               Texas| 34| 72|  3| 24|
|        North Dakota|  1|  1|  3|  2|
|        Pennsylvania|  3| 36| 18|  4|
|         Connecticut|  5|  0|  8|  2|
|             Vermont|  1|  1|  5| 16|
|            Nebraska|  1|  3|  0|  2|
|              Nevada|  2|  5|  0|  2|
|         Puerto Rico|  4|  0|  1|  1|
|          Washington|  6|  2| 15| 20|
|            Illinois| 32|  1| 14|  2|
|            Oklahoma| 18|  2|  0|  2|
|      Virgin Islands|  0|  0|  6|  0|
|            Delaware|  0|  0|  2|  4